https://adventofcode.com/2022/day/24

In [1]:
import numpy as np
import math
from functools import reduce
from heapq import heappush, heappop

In [2]:
with open("data/24.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#
"""

In [34]:
def blizzard_list(data):
    lines = data.strip().splitlines()
    h, w = len(lines) - 2, len(lines[0]) - 2
    north, south, east, west, empty = [np.zeros((h, w), dtype=bool) for _ in range(5)]
    drxnmap = {
        "^": north,
        "v": south,
        ">": east,
        "<": west,
        ".": empty,
    }
    for row, line in enumerate(lines[1:-1]):
        for col, char in enumerate(line[1:-1]):
            drxnmap[char][row, col] = True
    del empty

    header = np.ones((1, w), dtype=bool)
    header[0, 0] = False
    footer = np.ones((1, w), dtype=bool)
    footer[0, -1] = False

    L = []
    for _ in range(math.lcm(h, w)):
        bliz = reduce(np.logical_or, [north, south, east, west])
        L.append(np.vstack([header, bliz, footer]))

        north = np.vstack([north[1:], north[:1]])
        south = np.vstack([south[-1:], south[:-1]])
        east = np.hstack([east[:, -1:], east[:, :-1]])
        west = np.hstack([west[:, 1:], west[:, :1]])

    return L


def open_spots(coords, ar):
    r0, c0 = coords
    L = []
    for (r, c) in [(r0, c0), (r0+1, c0), (r0-1, c0), (r0, c0+1), (r0, c0-1)]:
        if r < 0 or c < 0:
            continue
        try:
            v = ar[r, c]
        except IndexError:
            pass
        else:
            if not v:
                L.append((r, c))
    return L


def dijkstra(data, start=(0, 0), finish=None, initial_state=0):
    states = blizzard_list(data)
    if finish is None:
        h, w = states[0].shape
        finish = (h-1, w-1)
    visited = {(start, initial_state): 0}
    pq = []
    heappush(pq, (0, start, initial_state))
    while pq:
        (d, p, s) = heappop(pq)
        if p == finish:
            return d
        next_s = (s + 1) % len(states)
        next_d = d + 1
        possibles = open_spots(p, states[next_s])
        if not possibles: # don't ever return to this node
            visited[(p, s)] = 0
            continue
        for poss in possibles:
            v = visited.get((poss, next_s))
            if v is None or v > next_d:
                visited[(poss, next_s)] = next_d
                heappush(pq, (next_d, poss, next_s))

In [36]:
%%time
dijkstra(testdata)

CPU times: user 4.52 ms, sys: 277 µs, total: 4.8 ms
Wall time: 3.33 ms


18

Part 1

In [37]:
%%time
dijkstra(data)

CPU times: user 476 ms, sys: 3.85 ms, total: 480 ms
Wall time: 479 ms


225

Part 2

In [38]:
test_state = blizzard_list(testdata)[0]
h, w = test_state.shape
mod = math.lcm(h-2, w)
finish = (h-1, w-1)
mod, finish

(12, (5, 5))

In [39]:
first_leg = 18  # from part 1
initial_state = first_leg % mod
dijkstra(testdata, start=finish, finish=(0, 0), initial_state=initial_state)

23

In [40]:
two_legs = 18 + 23
initial_state = two_legs % mod
dijkstra(testdata, start=(0,0), finish=finish, initial_state=initial_state)

13

In [41]:
18 + 23 + 13

54

In [42]:
state = blizzard_list(data)[0]
h, w = state.shape
mod = math.lcm(h-2, w)
finish = (h-1, w-1)
mod, finish

(700, (36, 99))

In [43]:
first_leg = 225  # from part 1
initial_state = first_leg % mod
dijkstra(data, start=finish, finish=(0, 0), initial_state=initial_state)

238

In [44]:
two_legs = 225 + 238
initial_state = two_legs % mod
dijkstra(data, start=(0,0), finish=finish, initial_state=initial_state)

248

In [45]:
225 + 238 + 248

711